In [15]:
import os
import json
import cv2
import matplotlib.pyplot as plt
import shutil
import pandas as pd
from tqdm import tqdm

In [16]:
cwd = os.getcwd()
fathomnet_path = os.path.abspath(os.path.join(cwd, '..'))
datasets_path = os.path.abspath(os.path.join(cwd, '..', 'datasets'))
print(fathomnet_path)
print(datasets_path)

/media/jorrit/Storage SSD/fathomnet
/media/jorrit/Storage SSD/fathomnet/datasets


In [17]:
# os.chdir(datasets_path)
# # os.makedirs(os.path.join(datasets_path, 'FN_23_coco/img_train'), exist_ok=True)
# # os.makedirs(os.path.join(datasets_path, 'FN_23_coco/img_eval'), exist_ok=True)
# os.makedirs(os.path.join(datasets_path, 'FN_23_yolo/train/images'), exist_ok=True)
# os.makedirs(os.path.join(datasets_path, 'FN_23_yolo/train/labels'), exist_ok=True)
# os.makedirs(os.path.join(datasets_path, 'FN_23_yolo/valid/images'), exist_ok=True)
# os.makedirs(os.path.join(datasets_path, 'FN_23_yolo/valid/labels'), exist_ok=True)
# os.makedirs(os.path.join(datasets_path, 'FN_23_yolo/test/images'), exist_ok=True)
# os.makedirs(os.path.join(datasets_path, 'FN_23_yolo/test/labels'), exist_ok=True)

In [18]:
with open(os.path.join(fathomnet_path, 'object_detection', 'train.json')) as f:
    dict_train = json.load(f)
with open(os.path.join(fathomnet_path, 'object_detection', 'eval.json')) as f:
    dict_eval = json.load(f)

print("images in /train.json", len(dict_train["images"]))
print("images in /eval.json", len(dict_eval["images"]))

# CHOOSE ANNOTATION_DICT: dict_train, dict_eval
annotation_dict = dict_eval

images in /train.json 5950
images in /eval.json 10744


In [19]:
# Inspect structure annotation_dict
print(annotation_dict.keys())
print(annotation_dict['info'])
# print(annotation_dict['licenses']) # only dict_train
print(annotation_dict['images'][0])
# print(annotation_dict['annotations'][0]) # only dict_train
# print(annotation_dict['categories'][0]) # only dict_train
# pd.DataFrame(annotation_dict["images"])

dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])
{'year': 2023, 'version': '0', 'description': 'Generated by FathomNet', 'contributor': 'FathomNet', 'url': 'https://fathomnet.org', 'date_created': '2023/03/20'}
{'id': 1, 'width': 1920, 'height': 1079, 'file_name': '4172b65a-8b4f-4175-94e4-00187e2d5f33.png', 'license': 0, 'flickr_url': 'https://fathomnet.org/static/m3/framegrabs/Mini%20ROV/images/145/772004--372cce99-76c2-44d4-a67f-687930fafa6a.png', 'coco_url': 'https://fathomnet.org/static/m3/framegrabs/Mini%20ROV/images/145/772004--372cce99-76c2-44d4-a67f-687930fafa6a.png', 'date_captured': '2019-09-24 12:11:05'}


In [20]:
output_path_train = os.path.join(datasets_path, 'FN_23_yolo/train/') 
output_path_valid = os.path.join(datasets_path, 'FN_23_yolo/valid/') 
output_path_test = os.path.join(datasets_path, 'FN_23_yolo/test/')  

# CHOOSE OUTPUT_PATH: train, valid, test
output_path = output_path_valid #.replace('\\', '/')
print('output_path = ' + output_path)

output_path = /media/jorrit/Storage SSD/fathomnet/datasets/FN_23_yolo/valid/


In [21]:
img_train = 'FN_23_coco/img_train/'
img_eval = 'FN_23_coco/img_eval/'

# CHOOSE HERE: train or eval images to download
img_folder = os.path.join(datasets_path, img_eval)

In [22]:
print(len(annotation_dict['images']))
print(output_path)
print(img_folder)

10744
/media/jorrit/Storage SSD/fathomnet/datasets/FN_23_yolo/valid/
/media/jorrit/Storage SSD/fathomnet/datasets/FN_23_coco/img_eval/


In [23]:
count = 0
for filename in tqdm(os.listdir(img_folder)):
    source = os.path.join(img_folder, filename)
    destination = f"{output_path}images/{filename}" # replaced /img{count}.jpg 
    
    if os.path.exists(source):
        shutil.copy(source, destination)
        count += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10744/10744 [04:38<00:00, 38.54it/s]


In [11]:
def get_img_ann(image_id):
    img_ann = []
    isFound = False
    for ann in annotation_dict['annotations']:
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    if isFound:
        return img_ann
    else:
        return None

In [12]:
def get_img(filename):
    for img in annotation_dict['images']:
        if img['file_name'] == filename:
            return img

In [14]:
# img_eval does not contain annotations

count = 0

# Jorrit changed to loop over the annotation files instead of over filename list []. 
for filename in tqdm(os.listdir(img_folder)):
    img = get_img(filename)
    img_id = img['id']
    img_w = img['width']
    img_h = img['height']

    img_ann = get_img_ann(img_id)

    if img_ann:
        filename_without_ext = os.path.splitext(filename)[0] # remove extention .png
        file_object = open(f"{output_path}labels/{filename_without_ext}.txt", "a") # replaced /img{count}.txt 
  

        for ann in img_ann:
            # Jorrit added an if statement to check if the annotation contains a bounding box
            if "bbox" in ann:
                current_category = ann['category_id'] # As yolo format labels start from 0 
#                 remap_category = subset2remap[current_category] # J. remap the subset class value to the remaped class value
                current_bbox = ann['bbox']
                x = current_bbox[0]
                y = current_bbox[1]
                w = current_bbox[2]
                h = current_bbox[3]

                x_centre = (x + (x+w))/2 # Finding midpoints
                y_centre = (y + (y+h))/2

                x_centre = x_centre / img_w # Normalization
                y_centre = y_centre / img_h
                w = w / img_w
                h = h / img_h

                x_centre = format(x_centre, '.6f') # Limiting upto fix number of decimal places
                y_centre = format(y_centre, '.6f')
                w = format(w, '.6f')
                h = format(h, '.6f')

                file_object.write(f"{current_category} {x_centre} {y_centre} {w} {h}\n")  # J. changed current_category to remap_category

        file_object.close()
        count += 1  # This should be outside the if img_ann block.

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5950/5950 [00:14<00:00, 405.53it/s]


In [ ]:
# after converted all images + labels to yolo format run the code below in the terminal and adapt it to your preference. 
# python train.py --data CCT20.yaml --epochs 3 --weights '' --cfg yolov5n.yaml  --batch-size 2